In [1]:
import cv2
import os
import timm
import numpy as np
import pandas as pd
import albumentations as A
import logging
import json
from datetime import datetime

from glob import glob
from tqdm import tqdm
from easydict import EasyDict
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.tensorboard import SummaryWriter

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

# Dataset

In [2]:
class CustomDataset(Dataset):
    def __init__(self, img_list, label_list=None, transforms=None) :
        self.img_list = img_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path = self.img_list[idx]
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
        if self.transforms:            
            img = self.transforms(image=img)['image']
        
        # training
        if self.label_list is not None :
            label = self.label_list[idx]
            return img, torch.tensor(label)
        
        # test
        else :
            return img

# Focal Loss

In [3]:
class FocalLoss(nn.Module) :
    def __init__(self, alpha=2, gamma=2, logits=False, reduction='none') :
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduction = reduction

    def forward(self, inputs, targets) :
        ce_loss = nn.CrossEntropyLoss(reduction=self.reduction)(inputs, targets)
        pt = torch.exp(-ce_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * ce_loss

        if self.reduction :
            return torch.mean(F_loss)
        else :
            return F_loss

# MODEL

In [4]:
class CNN(nn.Module):
    def __init__(self, model_name) :
        super().__init__()
        self.model = timm.create_model(model_name=model_name, num_classes=10, pretrained=True)
        
    def forward(self, x) :
        x = self.model(x)            
        return x

# Augmentation

In [5]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int32(W * cut_rat)
    cut_h = np.int32(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix(imgs, labels):
    lam = np.random.beta(1.0, 1.0)
    rand_index = torch.randperm(imgs.size()[0]).cuda()
    target_a = labels
    target_b = labels[rand_index]
    bbx1, bby1, bbx2, bby2 = rand_bbox(imgs.size(), lam)
    imgs[:, :, bbx1:bbx2, bby1:bby2] = imgs[rand_index, :, bbx1:bbx2, bby1:bby2]

    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (imgs.size()[-1] * imgs.size()[-2]))

    return imgs, lam, target_a, target_b

def mixup(imgs, labels) :
    lam = np.random.beta(1.0, 1.0)
    rand_index = torch.randperm(imgs.size()[0]).cuda()
    mixed_imgs = lam * imgs + (1 - lam) * imgs[rand_index, :]
    target_a, target_b = labels, labels[rand_index]
    
    return mixed_imgs, lam, target_a, target_b

# Training

In [6]:
def weight_freeze(model) :
    for i, child in enumerate(model.children()) :
        for n, p in child.named_modules() :
            if n != 'classifier' :
                for param in p.parameters():
                    param.requires_grad = False    
            elif n == 'classifier' : 
                for param in p.parameters():
                    param.requires_grad = True
    return model

In [7]:
def weight_load(model, optimizer, ckpt):
    checkpoint = torch.load(ckpt)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    if CFG['clssifier_freeze'] :
        model = weight_freeze(model)
        
    return model, optimizer, checkpoint['epoch']

In [8]:
def score(true_labels, model_preds) :
    model_preds = model_preds.argmax(1).detach().cpu().numpy().tolist()
    true_labels = true_labels.detach().cpu().numpy().tolist()
    return accuracy_score(true_labels, model_preds)

In [9]:
def logging(logger, data, step) :
    for i, (k,v) in enumerate(data.items()) :
        logger.add_scalar(k, v, step)
    #return logger

In [10]:
def img_parser(data_path, div, training=True) :
    path = sorted(glob(data_path), key = lambda x : int(x.split('\\')[-1].split('.')[0]))   
    
    if training:    
        return path[:div], path[div:]    
    else :
        return path

In [11]:
def transform_parser(grid_shuffle_p=0.8) :
    return A.Compose([
        A.Rotate(limit=(45), p=1),
        A.RandomGridShuffle(p=grid_shuffle_p, grid=(2,2)),
        A.Normalize(),
        ToTensorV2()
    ])

In [12]:
def image_label_dataset(df_path, div=0.8, grid_shuffle_p=0.8, training=True) :
    all_df = pd.read_csv(df_path)
    transform = transform_parser(grid_shuffle_p=grid_shuffle_p)
    
    if training :
        train_df = all_df.iloc[:int(len(all_df)*div)]
        val_df = all_df.iloc[int(len(all_df)*div):]
        
        train_img, valid_img = img_parser(CFG['DATA_PATH'], int(len(all_df)*div), training)
        return (train_img, valid_img), (train_df['label'].values, val_df['label'].values), transform
    
    else :
        img = img_parser(CFG['DATA_PATH'], div=None, training=training)
        return np.array(img), all_df['label'].values, transform   

In [13]:
# def custom_dataload(df_path='./data/train.csv', div=0.8, grid_shuffle_p=0.8):
#     all_df = pd.read_csv(df_path)

#     train_df = all_df.iloc[:int(len(all_df)*div)]
#     val_df = all_df.iloc[int(len(all_df)*div):]

#     train_img, valid_img = img_parser(CFG['DATA_PATH'], int(len(all_df)*div))

#     transform = transform_parser(grid_shuffle_p=grid_shuffle_p)

#     train_dataset = CustomDataset(train_img, train_df['label'].values, transform)
#     train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

#     val_dataset = CustomDataset(valid_img, val_df['label'].values, transform)
#     val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
#     return train_loader, val_loader

In [14]:
def custom_dataload(img_set, label_set, transform, shuffle) :
    ds = CustomDataset(img_set, label_set, transform)
    dl = DataLoader(ds, batch_size = CFG['BATCH_SIZE'], shuffle=shuffle, num_workers=0)    
    return dl

In [15]:
def train_and_valid_dataload(img_set, label_set, transform) :   
    train_loader = custom_dataload(img_set[0], label_set[0], transform, True)
    val_loader = custom_dataload(img_set[1], label_set[1], transform, False)
    return train_loader, val_loader

In [16]:
def validation(model, criterion, val_loader, device, log_writter):
    model.eval()
    true_labels = []
    model_preds = []
    val_loss = []
    with torch.no_grad():
        for img, label in tqdm(iter(val_loader)):
            img = img.to(device)
            label = label.to(device)

            model_pred = model(img)
            loss = criterion(model_pred, label)

            val_loss.append(loss.item())

            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()            
            
    return np.mean(val_loss), accuracy_score(true_labels, model_preds)

In [17]:
def training(model, optimizer, criterion, data_set, device) :
    
    train_loader ,val_loader = train_and_valid_dataload(data_set[0], data_set[1], transform)
        
    if CFG['reuse'] : 
        model, optimizer, start_epoch = weight_load(model, optimizer, CFG['checkpoint'])
    else : 
        start_epoch = 0
        
    # tensorboard
    log_writter = SummaryWriter(CFG['LOG'])
    best_score = 0
    apply_mixup = False
    for epoch in range(start_epoch + 1, CFG['EPOCHS'] + 1) :
        
        if epoch == CFG['reg_step'][0] :
            train_loader, val_loader = train_and_valid_dataload(data_set[0], 
                                                                data_set[1], 
                                                                transform_parser(grid_shuffle_p=0))
            apply_mixup = True
            
        elif epoch == CFG['reg_step'][1] :
            apply_mixup = False
            model = weight_freeze(model)
        
        tqdm_train = tqdm(train_loader)
        train_acc, train_loss = [], []
        
        for batch, (img, label) in enumerate(tqdm_train, start=1) :
            model.train()
            optimizer.zero_grad()
            
            img = img.to(device)
            label = label.to(device)
            
            if apply_mixup :
                img, lam, label_a, label_b = mixup(img, label)
                output = model(img)
                loss = lam * criterion(output, label_a) + (1-lam) * criterion(output, label_b)
            else :
                output = model(img)
                loss = criterion(output, label)
            
            loss.backward()
            optimizer.step()
            
            acc = score(label, output)
            train_acc.append(acc)
            train_loss.append(loss.item())
            
            tqdm_train.set_postfix({
                'Training Acc' : np.mean(train_acc),
                'Training Loss' : np.mean(train_loss)
            })
            
            data = {
                'training loss' : loss.item(),
                'training acc' : acc
            }
            logging(log_writter, data, epoch * len(train_loader) + batch)
        
        # validation
        val_loss, val_acc = validation(model, criterion, val_loader, device, log_writter)
        
        data = {
            'validation loss' : val_loss,
            'validation acc' : val_acc
        }
        logging(log_writter, data, epoch)
        print(f'Epoch : [{epoch}] Val Loss : [{val_loss}] Val ACC : [{val_acc}]')
        
        if best_score < val_acc:
            best_score = val_acc
            torch.save({
                    "epoch" : epoch,
                    "model_state_dict" : model.state_dict(),
                    "optimizer_state_dict" : optimizer.state_dict()
                }, './ckpt/'+str(epoch)+'E-val'+str(best_score)+'-'+CFG['output'])

In [18]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

CFG = {
    'EPOCHS':70,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':64,
    'DATA_PATH' : './data/img/224img_train/*',
    
    'output' : 'scratch-weigt_freeze10E-mixup25E-grid_shuffle35E-focal-effib0.pth',
    'LOG' : "./tensorboard/PCA_img/weigt_freeze10E-mixup25E-grid_shuffle35E",   
    'reg_step' : [36, 61],
    'focal_gamma' : 2,
    'focal_alpha' : 2,
    
    'kfold' : 5,
    
    'model_name' : 'efficientnet_b0',
    
    'reuse': False,
    'clssifier_freeze' : False,
    'checkpoint' : './ckpt/23E-val0.9369-scratch-weigt_freeze5E-mixup15E-grid_shuffle25E-focal-effib0.pth',

}


In [19]:
cfg_save_name = datetime.now().strftime('%Y_%m_%d-%H_%M_%S') + '_' +CFG['output'].split('.')[0]
with open(f"./ckpt/{cfg_save_name}.json", 'w') as f :
    json.dump(CFG, f, indent="\t")

In [20]:
img_set, label_set, transform = image_label_dataset(df_path='./data/train.csv', div=0.8, grid_shuffle_p=0.8, training=False)
kfold = StratifiedKFold(n_splits=CFG['kfold'], shuffle=True)
for k, (train_ind, valid_ind) in enumerate(kfold.split(img_set, label_set)) :
    model = CNN(CFG['model_name']).to(device)
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=CFG['LEARNING_RATE'])
    criterion = FocalLoss(CFG['focal_alpha'], CFG['focal_gamma'])
    
    CFG['output'] = str(k) + 'fold_' + CFG['output']
    CFG['LOG'] = CFG['LOG'] + '_' + str(k) + 'fold'
    
    data_set=((img_set[train_ind], img_set[valid_ind]), 
              (label_set[train_ind], label_set[valid_ind]))
    
    print(f"======== {k}fold Training ========")
    training(model, optimizer, criterion, data_set, device)    

======== 0fold Training ========


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:36<00:00,  4.34it/s]


Epoch : [1] Val Loss : [0.7431151882098739] Val ACC : [0.7835]


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:31<00:00,  4.96it/s]


Epoch : [2] Val Loss : [0.48840411102316184] Val ACC : [0.8455]


...

Epoch : [68] Val Loss : [0.1055156599231966] Val ACC : [0.9592]


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:24<00:00,  6.36it/s]


Epoch : [69] Val Loss : [0.1042022155538486] Val ACC : [0.9611]


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:24<00:00,  6.37it/s]

Epoch : [70] Val Loss : [0.10042011639649037] Val ACC : [0.9607]


In [21]:
img_set, label_set, transform = image_label_dataset(df_path='./data/train.csv', div=0.8, grid_shuffle_p=0.8, training=True)
# train_loader ,val_loader = train_and_valid_dataload(img_set, label_set, transform)

In [22]:
model = CNN(CFG['model_name']).to(device)
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=CFG['LEARNING_RATE'])
criterion = FocalLoss(CFG['focal_alpha'], CFG['focal_gamma'])

In [24]:
training(model, optimizer, criterion, 
         ((img_set[0], img_set[1]), (label_set[0], label_set[1])),
         device
        )

1 E pass 
2 E pass 
3 E pass 
4 E pass 
5 E pass 
6 E pass 
7 E pass 
8 E pass 
9 E pass 
10 E pass 
11 E pass 
12 E pass 
13 E pass 
14 E pass 
15 E pass 
16 E pass 
17 E pass 
18 E pass 
19 E pass 
20 E pass 
21 E pass 
22 E pass 
23 E pass 
24 E pass 
25 E pass 
26 E pass 
27 E pass 
28 E pass 
29 E pass 
30 E pass 
31 E pass 
32 E pass 
33 E pass 
34 E pass 
35 E pass 


  2%|▉                                        | 14/625 [00:05<04:00,  2.54it/s, Training Acc=0.161, Training Loss=6.19]


KeyboardInterrupt: 

## submission

In [20]:
test_df = pd.read_csv('./data/sample_submission.csv')

test_img = img_parser('./data/img/224img_test/*', None, training=False)
transform = A.Compose([
        A.Rotate(limit=(45), p=1),
#         A.RandomGridShuffle(p=0.8, grid=(2,2)),
        A.Normalize(),
        ToTensorV2()
    ])

test_dataset = CustomDataset(test_img, None, transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

checkpoint = torch.load('./ckpt/64E-val0.9574-scratch-weigt_freeze10E-mixup25E-grid_shuffle35E-focal-effib0.pth')
model = CNN(CFG['model_name']).to(device)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [21]:
def predict(model, test_loader, device):
    model.to(device)
    model.eval()
    model_preds = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.to(device)
            
            batch_pred = model(img)
            
            model_preds += batch_pred.argmax(1).detach().cpu().numpy().tolist()
    
    return model_preds

In [22]:
preds = predict(model, test_loader, device)

100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [02:12<00:00,  4.73it/s]


In [23]:
test_df['label'] = preds

test_df.to_csv('./submission/64E-val0.9574-scratch-weigt_freeze10E-mixup25E-grid_shuffle35E-focal-effib0.csv', index=False)